In [1]:
import numpy as np
import pandas as pd

In [2]:
pred = np.load('./results/20230711_190414_dm64_df32_el2_topk3_nk6_fq_h_Exp/pred.npy') # (D,L,K)

In [3]:
D, L, K = pred.shape
pred = pred.reshape(D * L, K) # (D*L,K)

# Define the start and end times of each range
ranges_5min = [('05:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

# Create the DatetimeIndex from the list of values
datetime_5min_test_ids_full = pd.DatetimeIndex(index_values)

pred_df = pd.DataFrame(data=pred, index=datetime_5min_test_ids_full)


# pred_time_range
pred_ranges_5min = [('08:00:00', '08:55:00'), ('12:30:00', '13:25:00'), ('17:00:00', '17:55:00')]

# Define the start and end dates for the DatetimeIndex
start_date = '2023-07-01'
end_date = '2023-07-07'

# Create an empty list to store the DatetimeIndex values
index_values = []

# Loop through each date
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    # Loop through each range and generate the DatetimeIndex values within that range for the current date
    for start_time, end_time in pred_ranges_5min:
        index_values += pd.date_range(start=f'{date.date()} {start_time}', end=f'{date.date()} {end_time}', freq='5min', inclusive='both').tolist()

datetime_5min_test_ids = pd.DatetimeIndex(index_values)

pred_df = pred_df.loc[datetime_5min_test_ids]

In [4]:
result_flow_pred = np.sum(pred_df.values[:,:40].reshape((252,10,4)), axis=2)
result_speed_pred = np.round(np.sum((pred_df.values[:,:40] * pred_df.values[:,40:]).reshape((252,10,4)), axis=2) / result_flow_pred, decimals=2)
result_flow_pred = np.round(result_flow_pred, decimals=0)
final_results = np.hstack((result_flow_pred, result_speed_pred))

In [5]:
final_results.shape

(252, 20)

In [7]:
template_df = pd.read_csv('./submit_template.csv',index_col=['Time', 'Week'])

In [ ]:
template_df.iloc[:,:] = final_results

In [ ]:
template_df.iloc[:, :10] = template_df.iloc[:, :10].astype(int)
template_df.iloc[:, 10:] = template_df.iloc[:, 10:].astype(float).applymap("{:.2f}".format)

In [ ]:
template_df.to_csv('./results/submit.csv', index=True)